In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

# from CNN_LSTM_20201103-150358_0.4101.70, do ensemble

## Load and prepare data

In [2]:
train_file = 'example/train.csv'
val_file = 'example/val.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_seq': 40,
    'min_freq': 0,
    'batch_size': 64,
    'pretrained_file': pretrained_file
}


In [3]:
AttractiveData = AttractiveData(train_file, val_file, test_file, pretrained_file, config)

In [4]:
# for i, sentence in enumerate(AttractiveData.test_data):
#     if i == 3:
#         print(vars(AttractiveData.train_data[i]), vars(sentence))

In [5]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [6]:
max_len = 0
a = AttractiveData.train_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

38

## Start to train

In [7]:
config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'CNN_LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 8
config['hidden_dim'] = 30
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 200
config['lr'] = {
    'encoder': 1e-5,
    'embedding': 6e-6,
    'linear': 1e-5
}
config['num_layers'] = 1
config['kernel_size'] = 3
config['dropout'] = 0.1
config['train_len'] = AttractiveData.train_len
config['val_len'] = AttractiveData.val_len
config['test_len'] = AttractiveData.test_len

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([11857, 300])


In [8]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, AttractiveData.valloader, pretrained_embeddings)

In [ ]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

In [ ]:
AttractiveTrainer.train()

## for classification, not better

## Below is testing

In [9]:
from attractivenet import AttractiveNet

PATH = './model/CNN_LSTM_20201103-184418_0.4154.55'
# PATH = './model/CNN_LSTM_20201103-183552_0.4084.65'
# PATH = './model/CNN_LSTM_20201103-182128_0.4007.65'
# PATH = './model/CNN_LSTM_20201103-181740_0.4171.60'
# PATH = './model/CNN_LSTM_20201103-181044_0.4284.50'
# PATH = './model/CNN_LSTM_20201103-174348_0.4127.70'
# PATH = './model/CNN_LSTM_20201103-163042_0.4105.65'
# PATH = './model/CNN_LSTM_20201103-160026_0.4157.65'
# PATH = './model/CNN_LSTM_20201103-150358_0.4101.70'

load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(11857, 300, padding_idx=1)
  )
  (bigramcnn): Sequential(
    (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
  )
  (trigramcnn): Sequential(
    (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
  )
  (encoder_bigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (encoder_trigram): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
  (linear): Sequential(
    (0): Linear(in_features=120, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=1, bias=True)
  )
)

In [10]:
def predict_attractive(sentence, category, phase):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(0)
    # print(tensor_sentence.shape)

    prediction = load_model(tensor_sentence, tensor_category, phase=phase)

    return prediction

In [11]:
# train mean = 3.15, test mean = 2.8
predict_list = []
with torch.no_grad():
    for i, sentence in enumerate(AttractiveData.test_data):
        prediction = predict_attractive(sentence.Headline, sentence.Category, 'test')
        predict_list.append(prediction.item())
        # predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

## Below for statistics

In [12]:
import statistics
from sklearn.metrics import mean_squared_error

In [13]:
# # train mean = 3.15, test mean = 2.8
# train_list = []
# for i, sentence in enumerate(AttractiveData.train_data):
#     prediction = predict_attractive(sentence.Headline, sentence.Category, 'train')
#     train_list.append(prediction.item())
#     # train_list.append(prediction.item())
# # print(train_list)
# mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

In [14]:
# train_list[0:5], pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list()[0:5]

In [15]:
# a = AttractiveData.df_train['Label'].to_list()
# statistics.mean(a), statistics.stdev(a)

In [16]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.836371272671065, 0.30469634100347953)

In [17]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.0046003655152889985, 2.8534801034325197, 0.2907379251644403)

In [17]:
mean_squared_error(pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list(), pd.read_csv('./predict/ensemble.csv').sort_values(['ID']).Label.to_list())

0.0